In [11]:
""" Street registry data transformation task.
    -----------------------------
    Notebook which imports the dataset,
    and executes the required transformations.
"""

import os
os.system('pip install apache-beam[interactive]') # uncomment for first instance
import apache_beam as beam
import csv
import json
from google.colab import files

In [12]:
def parse_csv(line, header):
    """ Parser function for all lines in CSV file,
    also requires a header to be specified as a list.
    """

    fields = next(csv.reader([line]))
    return dict(zip(header, fields))

In [13]:
def create_id(element):
    """ Property ID creation function. Appends the primary
    and secondary address fields to the postcode. Returns
    a tuple of the property ID and remaining fields.
    """

    prop_id = element['postcode'].replace(' ','_') + '_' + \
              element['paon'].replace(' ', '_') + '_' + \
              element['saon'].replace(' ', '_')
    return prop_id, element

In [14]:
def json_transform(entry):
    """ Transform function to collect all of the relevant
    elements across the dataset into a dictionary.
    """

    prop_id, element = entry
    trans_data = {
                # transaction data
                "trans_id": element['uuid'],
                "price": element['price'],
                "date": element['date'],
                "ppd": element['ppd'],
                "record": element['record'],
                # address data
                "postcode": element['postcode'],
                "paon": element['paon'],
                "saon": element['saon'],
                "street": element['street'],
                "locality": element['locality'],
                "town": element['town'],
                "district": element['district'],
                "country": element['country'],
                # property data
                "type": element['type'],
                "newbuild": element['newbuild'],
                "tenure": element['tenure']}

    return prop_id, trans_data

In [15]:
def create_json(entry):
    """ Create JSON structure ready for writing. Each entry
    corresponds to a single property with the relevant
    elements. Also contains an element transactions containing
    a list of all transactions and their relevant fields.
    """

    prop_id, trans_data = entry
    out_data = {
        "postcode": trans_data[0]['postcode'],
        "paon": trans_data[0]['paon'],
        "saon": trans_data[0]['saon'],
        "street": trans_data[0]['street'],
        "locality": trans_data[0]['locality'],
        "town": trans_data[0]['town'],
        "district": trans_data[0]['district'],
        "country": trans_data[0]['country'],
        "type": trans_data[0]['type'],
        "newbuild": trans_data[0]['newbuild'],
        "tenure": trans_data[0]['tenure'],
        "transactions": [
            {
                'trans_id': trans['trans_id'],
                'price': trans['price'],
                'date': trans['date'],
                'ppd': trans['ppd'],
                'record': trans['record']
            } for trans in trans_data]}

    out = json.dumps(out_data)
    return out

In [16]:
header = ['uuid','price','date','postcode','type','newbuild','tenure',
          'paon','saon','street','locality','town','district','country',
          'ppd','record']

#fname = 'pp-july24.csv'
fname = 'pp-2020-nohead.csv'

In [17]:
# beam pipeline instructions
with beam.Pipeline() as pipe:
    (
        pipe
        | 'read csv' >> beam.io.ReadFromText(fname)
        | 'parse csv' >> beam.Map(lambda line: parse_csv(line, header))
        | 'create property id' >> beam.Map(create_id)
        | 'transform to json' >> beam.Map(json_transform)
        | 'group by property id' >> beam.GroupByKey()
        | 'create json output' >> beam.Map(create_json)
        | 'write json' >> beam.io.WriteToText(fname[:-4], file_name_suffix='.json')
    )

In [18]:
files.download(fname[:-4]+'-00000-of-00001.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Header list and description:

| **Data item**                         | **Explanation (where appropriate)**                                                                                                                                                                   |
|---------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Transaction unique identifier**     | A reference number which is generated automatically recording each<br> published sale. The number is unique and will change<br> each time a sale is recorded.                                          |
| **Price**                             | Sale price stated on the transfer deed.                                                                                                                                                               |
| **Date of Transfer**                  | Date when the sale was completed, as stated on the<br> transfer deed.                                                                                                                                 |
| **Postcode**                          | This is the postcode used at the time of the original<br> transaction. Note that postcodes can be reallocated<br> and these changes are not reflected in the<br> Price Paid Dataset.                   |
| **Property Type**                     | D = Detached, S = Semi-Detached, T = Terraced, F = Flats/Maisonettes,<br> O = Other. Note that i) HM Land Registry only<br> records the above categories to describe property type,<br> ii) they do not separately identify bungalows,<br> iii) end-of-terrace properties are included in the<br> Terraced category, iii) 'Other' is only valid for<br> transactions relating to property types not covered by existing values. |
| **Old/New**                           | Indicates the age of the property and applies to all<br> price paid transactions, residential and non-residential.<br> Y = a newly built property, N = an established<br> residential building.         |
| **Duration**                          | Relates to the tenure: F = Freehold, L = Leasehold etc.<br> HM Land Registry does not record leases of 7 years<br> or less in the Price Paid Dataset.                                                 |
| **PAON**                              | Primary Addressable Object Name. Typically the<br> house number or name.                                                                                                                              |
| **SAON**                              | Secondary Addressable Object Name. Where a property<br> has been divided into separate units (for example,<br> flats), the PAON will identify the building and<br> a SAON will identify the flat/unit. |
| **Street**                            | Street name where the property is located.                                                                                                                                                            |
| **Locality**                          | The locality where the property is situated.                                                                                                                                                          |
| **Town/City**                         | The town or city of the property.                                                                                                                                                                     |
| **District**                          | The administrative district of the property.                                                                                                                                                          |
| **County**                            | The county where the property is located.                                                                                                                                                             |
| **PPD Category Type**                 | Indicates the type of Price Paid transaction. A = Standard Price<br> Paid entry, including single residential property sold for value.<br> B = Additional Price Paid entry including transfers under a power of sale/repossessions,<br> buy-to-lets (where they can be identified by a Mortgage),<br> and transfers to non-private individuals.<br> Category B does not separately identify transaction types stated.               |
| **Record Status - monthly file only** | Indicates additions, changes and deletions to the records.<br> A = Addition, C = Change, D = Delete. Note that where<br> a transaction changes category type due to misallocation<br> it will be deleted from the original category type<br> and added to the correct category with a new<br> transaction unique identifier. |


Note for improvement in pipeline:

There are some transactions which are linked to the original one but should not be considered a separate transaction. These will be recorded separately with a slightly different date and possibly price, but all other fields identical, except for the PPD category. The transaction pair will consist of a type A plus a type B in the PPD field.

Ideally in cases such as these, the earlier transaction will be filtered out as a duplicate.

Example:
YO8 8SH, 15 Willow Gardens (2020)